In [29]:
import torch
from transformers import AutoModel
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
from datasets import load_dataset

import random
import random
from PIL import ImageDraw, ImageFont, Image
import pathlib
import sklearn
import datasets
import pandas as pd
import sklearn.preprocessing
import sklearn.model_selection
import glob
import functools


In [60]:
dataset_path = '/Users/tylerklimas/Desktop/BERTModel/dataset_processed'
dataset_raw = datasets.load_from_disk(dataset_path)
dataset_raw['train'][0]['text']

"Gmall Men's Summer Face Cover UV Protection Neck Gaiter Scarf Sunscreen Breathable Bandana, Black, 1 Piece"

In [31]:
labels = dataset_raw['train'].features['label'].names
labels

['ACCESSORY',
 'BOOT',
 'CELLULAR_PHONE_CASE',
 'CHAIR',
 'EARRING',
 'FINEEARRING',
 'FINENECKLACEBRACELETANKLET',
 'FINERING',
 'GROCERY',
 'HANDBAG',
 'HARDWARE_HANDLE',
 'HAT',
 'HEALTH_PERSONAL_CARE',
 'HOME',
 'HOME_BED_AND_BATH',
 'HOME_FURNITURE_AND_DECOR',
 'JANITORIAL_SUPPLY',
 'KITCHEN',
 'LAMP',
 'LIGHT_BULB',
 'LIGHT_FIXTURE',
 'OFFICE_PRODUCTS',
 'OUTDOOR_LIVING',
 'PET_SUPPLIES',
 'RUG',
 'SANDAL',
 'SHOES',
 'SOFA',
 'SPORTING_GOODS',
 'TABLE',
 'WALL_ART']

In [32]:
idx2str = {}
str2idx = {}
for idx, ele in enumerate(labels):
    str2idx[ele] = idx
    idx2str[idx] = ele


In [34]:
base_model = "distilbert-base-uncased"
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
model = transformers.AutoModel.from_pretrained(base_model)

In [64]:
def tokenize(inpt):
    return tokenizer(inpt['text'], truncation=True, padding='max_length')

tokenized_datasets = dataset_raw.with_transform(tokenize)
# tokenize(dataset_raw['train'][0]) the text element gets transformed into integers 
# "Gmall Men's Summer Face Cover UV Protection Neck Gaiter Scarf Sunscreen Breathable Bandana, Black, 1 Piece"
# 101, 20917, 3363, 2273, 1005, 1055, 2621, 2227, 3104, 23068, 3860, 3300, 11721, 21646, 18982, 19352, 24410, 3052, 3085, 24112, 2532, 1010, 2304, 1010, 1015, 3538, 102

{'input_ids': [101, 20917, 3363, 2273, 1005, 1055, 2621, 2227, 3104, 23068, 3860, 3300, 11721, 21646, 18982, 19352, 24410, 3052, 3085, 24112, 2532, 1010, 2304, 1010, 1015, 3538, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

{'input_ids': [101,
  20917,
  3363,
  2273,
  1005,
  1055,
  2621,
  2227,
  3104,
  23068,
  3860,
  3300,
  11721,
  21646,
  18982,
  19352,
  24410,
  3052,
  3085,
  24112,
  2532,
  1010,
  2304,
  1010,
  1015,
  3538,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
